<a href="https://colab.research.google.com/github/Daramluv/Google_colab/blob/main/Transportation_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 34.8 MB/s eta 0:00:00


In [2]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('tomato shipping')

# indices for companies and routes
plant = [*range(0,3)]
customer = [*range(5,7)]
transit = [*range(3,5)]
node = [*range(0,7)]

# print(plant)
# print(customer)
# print(transit)


# shipping cost
c = [[0.0,	5.0,	3.0,	5.0,	5.0,	20.0,	20.0],
    [9.0,	0.0,	9.0,	1.0,	1.0,	8.0,	15.0],
    [0.4,	8.0,	0.0,	1.0,	0.5,	10.0,	12.0],
    [0.0,	0.0,	0.0,	0.0,	1.2,	2.0,	12.0],
    [0.0,	0.0,	0.0,	0.8,	0.0,	2.0,	12.0],
    [0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	1.0],
    [0.0,	0.0,	0.0,	0.0,	0.0,	7.0,	0.0]]

d = [0.0, 0.0, 0.0, 0.0, 0.0, 400.0, 180.0]

s = [200.0, 300.0, 100.0, 0.0, 0.0, 0.0, 0.0]

T = 200

# Valid set of tuples
A = []
for i in node:
    for j in node:
        if c[i][j] > 0:
            tp = i,j
            A.append(tp)

# take a look at the set
# print(np.matrix(A))

# valid set of inbound routes for node j
AI = []
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if j==k:
                    tp = i,j
                    A_temp.append(tp)
    AI.append(A_temp)
    k+=1

# take a look at a sample
# print(np.matrix(AI[0]))

# valid set of outbound routes for node j
AO = []
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if i==k:
                    tp = i,j
                    A_temp.append(tp)
    AO.append(A_temp)
    k+=1

# take a look at a sample
# print(np.matrix(AO[0]))

# Build decision variables: where to assign company i to route j
x = m.addVars(A, vtype=GRB.CONTINUOUS, name='Ship')

# Objective function: Minimize total shipment cost
m.setObjective(gp.quicksum(c[i][j]*x[(i,j)] for i,j in A), GRB.MINIMIZE)

# Satisfy customer demand
demandConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) >= d[j] for j in customer),
                                      name='demandConstrs')

# Cannot exceed plant capacity
supplyConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AO[i]) - gp.quicksum(x[(k,i)] for k,i in AI[i]) <= s[i] for i in plant),
                                      name='supplyConstrs')

# Blanacing for transit nodes
transitConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) == 0 for j in transit),
                                      name='transitConstrs')

# Shipment capacity constraint
capacityConstrs = m.addConstrs((x[(i,j)] <= T for i,j in A),
                                      name='capacityConstrs')

# Run optimization engine
m.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 33 rows, 26 columns and 78 nonzeros
Model fingerprint: 0x27e79db7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+02]
Presolve removed 26 rows and 0 columns
Presolve time: 0.01s
Presolved: 7 rows, 26 columns, 52 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.250000e+01   0.000000e+00      0s
       7    3.2600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.260000000e+03


In [4]:
#####################################################
#         Shipment results
#####################################################

print(f"\n\n___Optimal shipment from plants to customers________")
t_cost = 0
for i,j in A:
    if x[(i,j)].x > 0:
        if i<=2:
          s_node_type = "plant"
        elif i<=4:
          s_node_type = "warehouse"
        else:
          s_node_type = "customer"
        if j<=2:
          d_node_type = "plant"
        elif j<=4:
          d_node_type = "warehouse"
        else:
          d_node_type = "customer"

        print("Shipping %4d tons from %s %2d to %s %2d" % (x[(i,j)].x, s_node_type, i+1, d_node_type, j+1))
        t_cost += x[(i,j)].x*c[i][j]

print("The total cost of satisfying customer demand is $%5d" % (t_cost))




___Optimal shipment from plants to customers________
Shipping  180 tons from plant  1 to plant  3
Shipping  120 tons from plant  2 to warehouse  4
Shipping  180 tons from plant  2 to customer  6
Shipping   80 tons from plant  3 to warehouse  4
Shipping  200 tons from plant  3 to warehouse  5
Shipping  200 tons from warehouse  4 to customer  6
Shipping  200 tons from warehouse  5 to customer  6
Shipping  180 tons from customer  6 to customer  7
The total cost of satisfying customer demand is $ 3260


##  Conclusion

In this example, we addressed the tomato shipping problem. We determined the optimal shipment  of tomatos from plants to customers:
* Satisfy demand for each customer,
* Minimize the total shipping cost,  
* Ensure plant capacities are not exceeded, and
* Utilize transhipment to reduce shipping cost.

A special technique in the model formulation is sparse reprentation, where we significantly reduce the number of decision variables by restricting the set of decisions to be on the valid routes only. This benefit becomes more significant as problem size grows.

This tomato shipment model can be used in many different settings to help companies make informed decisions about satisfying customer demands from a set of plants where there are transit stations allowing for transhipments.
